This is dedicated to determining which users are pain patients and which conditions they have

In [13]:
import pandas as pd
#Plotting 
%matplotlib inline

from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

In [14]:
%cd twitterproject

# inject config value (on command line would've been --config=data-analysis)
import sys
sys.argv = ['data-analysis']
import environment

from TwitterDatabase.Repositories import DataRepositories as DR
from TwitterDatabase.DatabaseAccessObjects import DataConnections as DC
from TwitterDatabase.Models.WordORM import Word
from TwitterDatabase.Models.TweetORM import Users as User
from TwitterDatabase.Models.TweetORM import Tweet
from DataAnalysis.SearchTools.WordMaps import get_adjacent_word_counts, get_adjacent_words, get_user_ids_for_word

EXP_TERMS_FILEPATH = '%s/experimental-terms.xlsx' % environment.EXPERIMENTS_FOLDER
IDS_FILEPATH = "%s/temp_output/user-ids.xlsx" % environment.LOG_FOLDER_PATH


(bookmark:twitterproject) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterProject
/Users/adam/Dropbox/PainNarrativesLab/TwitterProject


# Find users whose profile contains an experimental term

In [ ]:

def get_rows_for_terms(wordFrame, experimentalTerms):
    return wordFrame[wordFrame.term.isin(experimentalTerms)]


def find_mapping(term, termMap):
    for t in termMap.T.index:
        if termMap[t].str.contains(term).any():
            return t
    return False


In [ ]:
# load in terms to search for
experimentalTerms = pd.read_excel(EXP_TERMS_FILEPATH, sheet_name='terms', squeeze=True)
termMap = pd.read_excel(EXP_TERMS_FILEPATH, sheet_name='mapping')

In [ ]:
%time

# iterate through each of the experimental terms, 
# getting user ids for each.

ids = []
for t in termMap.T.index:
    print(t)
    users = []
    for subterm in termMap[t]:
        users += [x[0] for x in get_user_ids_for_word(subterm)]
    users = list(set(users))
    ids.append(pd.Series(users, name=t))
    
ids = pd.DataFrame(ids)

# Save results
ids.T.to_excel(IDS_FILEPATH) 

len(ids)

# Counts of tweets belonging to users and broken out by condition

In [ ]:
test_user_id1 = 10712602
test_user_id2 = 7609402
test_tweet_id = 340493586317582340

In [ ]:
def get_tweet_count(userId):
    """Returns how many tweets we have for the given user id"""
    return dao.session.query(Tweet).filter(Tweet.userID == userId).count()

def get_tweets(userId):
    """Returns all tweets belonging to the user"""
    return dao.session.query(Tweet).filter(Tweet.userID == userId).all()

def get_tweet_timestamps(userId):
    """Returns the `created_at` field` for all tweets belonging to the given user """
    return [x.created_at for x in get_tweets(userId)]

def get_user(userId):
    """Returns the user object for the id"""
    return dao.session.query(User).filter(User.userID == userId).all()

## load data

In [ ]:
e = DC.initialize_engine('mysql')
dao = DC.DAO(e)

In [ ]:
# load userids from file
userIds = pd.read_excel(IDS_FILEPATH)

In [ ]:
# display counts for each condition
print("Number of user descriptions containing the term \n")
for r in userIds.columns:
    print(" %s: %s" % (r, len(userIds[r].dropna())))

## Calculate tweet counts for each condition

In [ ]:
counts = {}

for condition in userIds.columns:
    counts[condition] = []
    for user_id in userIds[condition].dropna():
        # look up the number of tweets that we've captured
        # by the user.
        r = (user_id, get_tweet_count(user_id))
        counts[condition].append(r) 

# yes, we're cheating with scope. Don't move these functions without
# ensuring that counts will still be available

def get_non_zero(condition):
    """Returns user ids for users who have used a targeted
    term in their profile, and who have tweeted at least once
    using a term which was picked up by the miner. 
    
    NB, this does not mean that the term in the tweet(s) was
    the same term found in their profile.
    
    Note that this is dependent on the particular scope
    """
    return [x for x in counts[condition] if x[1] >= 1]

def get_num_non_zero(condition):
    return len(get_non_zero(condition))

def get_num(condition):
    return len(counts[condition])

## Count how many users we have captured tweets for

This is needed because the miner searches for users with the term in their profile and tweets. It stores them in the same way

In [ ]:
non_zero_counts = {}

for condition in userIds.columns:
    non_zero_counts[condition] = {'descripts containing term' : get_num(condition), 'users with 1+ tweets': get_num_non_zero(condition)}

non_zero_counts = pd.DataFrame(non_zero_counts)

### Results

This means that 1439 users had the word 'arthritis' in their profile (twitter calls it a 'description'). However, we only have at least 1 tweet for 77 of the 1439. That's because the mining program's query returned both users with the term in their description and tweets containing the search term.

As part of our research, we will probably want to set up a new search which pulls in tweets for a sample of the remaining users. That should be done after we figure out how to filter out some of the bots and commercial accounts.

__Users with 1+ tweets__

In [ ]:
non_zero_counts.T

In [ ]:
fig, axes = plt.subplots(figsize=(10,5))
non_zero_counts.T.plot(kind='bar', ax=axes)
axes.set_title('Users with target terms in profile and captured terms in tweets')
fig.tight_layout()

__Distribution of the counts__

The x axis is 'Number of tweets'. Had trouble with the formatting

In [ ]:
non_empty = []
for cond in userIds.columns:
    c = pd.Series([x[1] for x in get_non_zero(cond)])
    if len(c) > 1:
        non_empty.append((cond, c))

fig, axes = plt.subplots(ncols=1, nrows=len(non_empty), figsize=(10,15))
i = 0
for cond, series in non_empty:
#     series.plot(kind='kde', title=cond, ax=axes[i])
    sns.violinplot(series, ax=axes[i]); axes[i].set_title(cond); #axes[i].set_x_label('# tweets')
    i +=1
fig.tight_layout()

# Temporal distribution of captured tweets

We should know whether the tweets containing terms of interest are evenly distributed over the capture period.

In [ ]:
def make_timestamp_frame(terms):
    """Creates the data we need """
    ts = []
    i = 1
    for term in terms:
        for u, c in get_non_zero(term):
            for t in get_tweet_timestamps(u):
                ts.append({ 'tweetTime' : pd.to_datetime(t), 'tweet': i, 'term' : term})
        i += 1
        print("%s : %s" % (term, len(ts)))
    ts = pd.DataFrame(ts)
    return ts

terms = [t for t in userIds.columns]

# load the data
tweet_timestamps = make_timestamp_frame(terms)

#### Plot helpers

In [ ]:
from bokeh.palettes import Spectral6, Category20, magma, inferno, viridis

def color_generator(num_colors, palette_function=viridis):
    """Returns a color from the relevant palette"""
    colorlist = palette_function(num_colors)
    for c in colorlist:
        yield c


In [ ]:
# Make a dictionary to hardcode in ticker
dd = {}
for i in range(1, len(userIds.columns) +1):
    dd[i] = userIds.columns[i-1]
dd

In [ ]:
def ticker():
    """Replaces the numeric y axis label with the correct term
    The dict seems to need to be hardcoded since bokeh
    messes with any args or values which seem like they should be 
    in scope"""
    dd = {
        1: 'crps',
        2: 'migraine',
        3: 'fibromyalgia',
        4: 'spoonie',
        5: 'vulvodynia',
        6: 'endometriosis',
        7: 'neuropathy',
        8: 'arthritis',
        9: 'rhem_arthritis',
        10: 'shingles',
        11: 'backpain',
        12: 'headache'
    }

    term = dd.get( tick )
    return "{}".format( term )


### Plot the tweet distributions for each condition

In [ ]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.commits import data
from bokeh.transform import jitter

from bokeh.plotting import *
from bokeh.models import FuncTickFormatter
from bokeh.models.tickers import FixedTicker


def plot_tweet_distributions(frame, terms, title='tweet frequencies'):
    colorgen = color_generator(len(terms))

    # initialize the notebook output
    output_notebook()

    # create a new plot with a title and axis labels
    p = figure(title=title, 
               x_axis_type="datetime", 
               plot_width=800, 
               plot_height=500, 
               x_axis_label='timestamp', 
               y_axis_label='term')


    for term in terms.columns:
        color = next(colorgen)
        source = ColumnDataSource(frame[frame.term == term])
        p.circle(x='tweetTime', 
                 y=jitter('tweet', width=0.5, range=p.y_range), 
                 fill_color=color, 
                 source=source, 
                 alpha=0.6
                )

    p.x_range.range_padding = 0
    p.ygrid.grid_line_color = None
    # p.legend.orientation = "horizontal"

    # limit the displayed tick locations to those corresponding to the 
    # terms in the dataframe 
    tick_locations = [x for x in range(1, len(terms) + 1)]
    p.yaxis.ticker = FixedTicker(ticks=tick_locations)

    # Now add the labels instead of the numbers to the y axis
    p.yaxis.formatter = FuncTickFormatter.from_py_func(ticker)

    # show the results
    show(p)

In [ ]:
terms = userIds.columns
plot_tweet_distributions(tweet_timestamps, terms)